In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_66Agree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_AllAgree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/README.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/License.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_75Agree.txt
/kaggle/input/sentiment-analysis-for-financial-news/FinancialPhraseBank/Sentences_50Agree.txt
/kaggle/input/twitter-entity-sentiment-analysis/twitter_validation.csv
/kaggle/input/twitter-entity-sentiment-analysis/twitter_training.csv


In [2]:
! pip install --upgrade spacy
! pip install --upgrade scikit-learn

  Obtaining dependency information for scikit-learn from https://files.pythonhosted.org/packages/d0/0b/26ad95cf0b747be967b15fb71a06f5ac67aba0fd2f9cd174de6edefc4674/scikit_learn-1.3.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 70.8 MB/s eta 0:00:00:00:01:01
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2


In [4]:
import spacy
from sklearn.utils import resample
from textblob import TextBlob
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.sentiment import SentimentIntensityAnalyzer
import gensim.downloader as api
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from plotly import plot
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs,init_notebook_mode,plot,iplot
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [5]:
data=pd.read_csv('/kaggle/input/sentiment-analysis-for-financial-news/all-data.csv',
                 encoding='ISO-8859-1',names=['category','text'])
data.head()

,category,text
0,neutral,"According to Gran , the company has no plans t..."
1,neutral,Technopolis plans to develop in stages an area...
2,negative,The international electronic industry company ...
3,positive,With the new production plant the company woul...
4,positive,According to the company 's updated strategy f...


In [6]:
data.category.value_counts()

category
neutral     2879
positive    1363
negative     604
Name: count, dtype: int64

In [7]:
data.isnull().sum()

category    0
text        0
dtype: int64

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4846 entries, 0 to 4845
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   category  4846 non-null   object
 1   text      4846 non-null   object
dtypes: object(2)
memory usage: 75.8+ KB


In [9]:
selected_cats=[cat for cat in data.category.unique() if cat!='positive']
norm_length=data.category.value_counts()['positive']

def data_normalizing(dframe,categories,norm_len):
    data_prepared=[data[data.category=='positive']]
    for cat in categories:
        data_cat=data[data.category==cat]
        data_extracted=resample(data_cat,replace=True,n_samples=norm_len,random_state=23)
        data_prepared.append(data_extracted)
    return pd.concat(data_prepared,axis=0).reset_index(drop=True)
data_norm=data_normalizing(data,selected_cats,norm_length)
data_norm.head()

,category,text
0,positive,With the new production plant the company woul...
1,positive,According to the company 's updated strategy f...
2,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...
3,positive,"For the last quarter of 2010 , Componenta 's n..."
4,positive,"In the third quarter of 2010 , net sales incre..."


In [70]:
nlp=spacy.load('en_core_web_md')
def stop_word_removing(text):
    doc=nlp(text)
    non_stopped=[token.lemma_ for token in doc if  not token.is_stop and not 
                 token.is_punct and not token.like_num]
    non_stop_text=' '.join(non_stopped)
    doc_non_stop=nlp(non_stop_text)
    not_oov_text=[token.lemma_ for token in doc_non_stop if not token.is_oov]
    return ' '.join(not_oov_text)
data_norm['non_stop']=data_norm.text.apply(stop_word_removing)

In [71]:
data_norm

,category,text,non_stop,category_num,polarity
0,positive,With the new production plant the company woul...,new production plant company increase capacity...,1,-0.064802
1,positive,According to the company 's updated strategy f...,accord company update strategy year Basware ta...,1,0.000000
2,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,finance growth aggressively pursue growth stra...,1,0.500000
3,positive,"For the last quarter of 2010 , Componenta 's n...",quarter net sale double m m period year early ...,1,0.000000
4,positive,"In the third quarter of 2010 , net sales incre...",quarter net sale increase EUR mn operating pro...,1,0.000000
...,...,...,...,...,...
4084,negative,The airline 's share price closed down slightl...,airline share price close slightly $ Helsinki,-1,-0.140741
4085,negative,A tinyurl link takes users to a scamming site ...,link take user scamming site promise user earn...,-1,0.325000
4086,negative,"Pretax loss totalled EUR 49.9 mn , compared to...",pretax loss total EUR mn compare loss EUR mn c...,-1,0.000000
4087,negative,Hobby Hall 's sales decrease 26 pct due to imp...,Hobby Hall sale decrease pct implement new inf...,-1,0.005682


In [63]:
nlp=spacy.load('en_core_web_md')
a=data_norm.non_stop.iloc[2]
doc=nlp(a)
non_stopped=[token.lemma_ for token in doc if  not token.is_stop and not 
                 token.is_punct and not token.like_num and not token.is_oov]
non_stopped

['finance',
 'growth',
 'aggressively',
 'pursue',
 'growth',
 'strategy',
 'increasingly',
 'focus',
 'technologically',
 'demand',
 'hdi',
 'print',
 'circuit',
 'board',
 'pcb']

In [37]:
cat_encode={'positive':1,'neutral':0,'negative':-1}
data_norm['category_num']=data_norm.category.map(cat_encode)
data_norm.head()

,category,text,non_stop,category_num
0,positive,With the new production plant the company woul...,new production plant company increase capacity...,1
1,positive,According to the company 's updated strategy f...,accord company update strategy year Basware ta...,1
2,positive,FINANCING OF ASPOCOMP 'S GROWTH Aspocomp is ag...,financing ASPOCOMP GROWTH Aspocomp aggressivel...,1
3,positive,"For the last quarter of 2010 , Componenta 's n...",quarter Componenta net sale double EUR131 m EU...,1
4,positive,"In the third quarter of 2010 , net sales incre...",quarter net sale increase EUR mn operate profi...,1


In [38]:
def polarity_subjectiv(text):
    blob=TextBlob(text)
    return blob.sentiment.polarity
data_norm['polarity']=data_norm.text.apply(polarity_subjectiv)

In [39]:
sentiment_polarity=np.stack([data_norm.category_num.values,data_norm.polarity.values]).T
category_num=data_norm.category_num.values.reshape(1,-1)
polarity=data_norm.polarity.values.reshape(1,-1)
cosine_similarity(category_num,polarity)

array([[0.30866179]])

In [40]:
vectorizer=TfidfVectorizer()
vect_corpus=vectorizer.fit_transform(data_norm.non_stop)
feature_names=vectorizer.get_feature_names_out()
for feature in feature_names[:10]:
    ind=vectorizer.vocabulary_.get(feature)
    print(f"{feature}: {vectorizer.idf_[ind]}")


00: 7.930005887916956
09: 8.623153068476903
10: 7.119075671700628
100mn: 7.524540779808792
1023: 8.623153068476903
109mn: 8.217687960368737
10mn: 8.623153068476903
117mn: 7.930005887916956
12: 7.706862336602747
14: 8.623153068476903


In [ ]:
train_X,test_X,train_y,test_y=train_test_split(
    vect_corpus.toarray(),
    data_norm.category_num,
    test_size=0.2,random_state=23,stratify=data_norm.category_num)
train_X.shape,train_y.shape,test_X.shape,test_y.shape

In [ ]:
nlp_gensim=api.load('glove-wiki-gigaword-200')

In [ ]:
nlp_spacy=spacy.load('en_core_web_lg')

def word_to_vector(text):
    tokens=nlp_spacy(text)
    preprocessed=[]
    for token in tokens:
        if not token.is_alpha or token.is_stop or token.is_punct:
            continue
        preprocessed.append(token.lemma_)
    if not preprocessed:
        return np.zeros(nlp_gensim.vector_size)
    return nlp_gensim.get_mean_vector(preprocessed)
data_norm['word_vec']=data_norm['text'].apply(lambda text: word_to_vector(text))
data_norm.head()

In [ ]:
train_X_gsm,test_X_gsm,train_y_gsm,test_y_gsm=train_test_split(
    data_norm.word_vec,
    data_norm.category_num,
    test_size=0.2,random_state=23,stratify=data_norm.category_num)
train_X_gsm.shape,train_y_gsm.shape,test_X_gsm.shape,test_y_gsm.shape
train_X_gsm,test_X_gsm=np.stack(train_X_gsm),np.stack(test_X_gsm)
min_max=MinMaxScaler()
train_X_scaled=min_max.fit_transform(train_X_gsm)
test_X_scaled=min_max.transform(test_X_gsm)

In [ ]:
nb=RandomForestClassifier(random_state=23)
nb.fit(train_X_scaled,train_y_gsm)
pred=nb.predict(test_X_scaled)
from sklearn.metrics import accuracy_score
accuracy_score(test_y_gsm,pred)

In [ ]:
train_X[train_X<0]

In [ ]:
data.text.iloc[0]

In [ ]:
nb=KNeighborsClassifier(n_neighbors=12)
nb.fit(train_X,train_y)
pred=nb.predict(test_X)
from sklearn.metrics import accuracy_score
accuracy_score(test_y,pred)

In [ ]:
nb=KNeighborsClassifier(n_neighbors=12)
nb.fit(train_X,train_y)
pred=nb.predict(test_X)
from sklearn.metrics import accuracy_score
accuracy_score(test_y,pred)

In [ ]:
data_norm.head()

In [ ]:
model_NB=MultinomialNB()
model_LR=LogisticRegression(random_state=23)
model_KNN=KNeighborsClassifier()
model_RF=RandomForestClassifier(random_state=23)
model_XGB=XGBClassifier()
model_SVM=SVC(random_state=23)
class Model_evaluation:
    def __init__(self,model,train_X=train_X,train_y=train_y,test_X=test_X,test_y=test_y):
        self.model=model
        self.train_X=train_X
        self.train_y=train_y
        self.test_X=test_X
        self.test_y=test_y
    def create_model(self):
        model=self.model
        if model==model_XGB: model.fit(self.train_X,self.train_y+1)
        else: model.fit(self.train_X,self.train_y)
        return model.predict(self.test_X)
    def evaluate_pred(self):
        pred=self.create_model()
        if self.model==model_XGB:
            return accuracy_score(self.test_y+1,pred)
        return accuracy_score(self.test_y,pred)
eval_NB=Model_evaluation(model_NB).evaluate_pred()
eval_LR=Model_evaluation(model_LR).evaluate_pred()
eval_KNN=Model_evaluation(model_KNN).evaluate_pred()
eval_RF=Model_evaluation(model_RF).evaluate_pred()
eval_XGB=Model_evaluation(model_XGB).evaluate_pred()
eval_SVM=Model_evaluation(model_SVM).evaluate_pred()

In [ ]:
eval_NB_gms=Model_evaluation(model_NB,train_X_scaled,train_y_gsm,test_X_scaled,test_y_gsm).evaluate_pred()
eval_LR_gms=Model_evaluation(model_LR,train_X_scaled,train_y_gsm,test_X_scaled,test_y_gsm).evaluate_pred()
eval_KNN_gms=Model_evaluation(model_KNN,train_X_scaled,train_y_gsm,test_X_scaled,test_y_gsm).evaluate_pred()
eval_RF_gms=Model_evaluation(model_RF,train_X_scaled,train_y_gsm,test_X_scaled,test_y_gsm).evaluate_pred()
eval_XGB_gms=Model_evaluation(model_XGB,train_X_gsm,train_y_gsm,test_X_scaled,test_y_gsm).evaluate_pred()
eval_SVM_gms=Model_evaluation(model_SVM,train_X_gsm,train_y_gsm,test_X_scaled,test_y_gsm).evaluate_pred()

In [ ]:
test_evals=np.array([eval_NB,eval_LR,eval_KNN,eval_RF,eval_XGB,eval_SVM])*100
test_evals_gsm=np.array([eval_NB_gms,eval_LR_gms,eval_KNN_gms,eval_RF_gms,eval_XGB_gms,eval_SVM_gms])*100
model_names=['NB','LR','KNN','RF','XGB','SVM']
layout=go.Layout(title='Model accuracy results (TF-IDF vs Gensim)',
                 xaxis=dict(title='Models',titlefont=dict(
                     family='Courier New',size=18,color='green')),
                 yaxis=dict(title='accuracy %',titlefont=dict(
                     family='Courier New',size=18,color='blue')) )
TF_IDF=go.Bar(x=model_names,y=test_evals, marker=dict(
        color=['#fc2020', '#1ac61d', '#2b84dd', '#ff8033','#8e35ff','#f5f042']))
Gensim=go.Bar(x=model_names,y=test_evals_gsm, marker=dict(
        color=['#fc2020', '#1ac61d', '#2b84dd', '#ff8033','#8e35ff','#f5f042'] ))
bar_plot=go.Figure(data=[TF_IDF,Gensim],layout=layout)
iplot(bar_plot)

In [ ]:
vector_corpus=vectorizer.transform(data_norm.non_stop)
models=[model_NB,model_LR,model_KNN,model_RF,model_XGB,model_SVM]
model_names=['Naive Bayes','Linear Regression','K-nearest nieghborhood',
            'Random Forest','XGBoost','Support Vector Machine']
def cross_valid_eval(model,model_name,vector_array=vector_corpus):
    y_value=data_norm.category_num
    if model_name=='XGBoost':
        y_value+=1
    cross_validator=cross_val_score(estimator=model,X=vector_array.toarray(),
                            y=y_value,scoring='accuracy',
                            cv=5,verbose=2)        
    print(f"{model_name} accuracy (cv=5): {np.round(cross_validator.mean(),3)}")
for model,model_name in zip(models,model_names):
    cross_valid_eval(model,model_name)

In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_hub as hub
import tensorflow_text as text
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense,Bidirectional,LSTM,Embedding,Dropout
from keras.layers import GlobalAveragePooling1D

In [ ]:
train_text,test_text,train_y,test_y=train_test_split(
    data_norm.text,data_norm.category_num,test_size=0.2,
    stratify=data_norm.category_num,random_state=23)

In [ ]:
data_tokenizer=Tokenizer()
data_tokenizer.fit_on_texts(data_norm.text)
total_words=len(data_tokenizer.word_index)+1
tokenizer=Tokenizer()
tokenizer.fit_on_texts(train_text)
train_sequence=tokenizer.texts_to_sequences(train_text)
train_pad_seq=pad_sequences(train_sequence,padding='pre',truncating='pre')
max_length=train_pad_seq.shape[-1]
test_sequence=tokenizer.texts_to_sequences(test_text)
test_pad_seq=pad_sequences(test_sequence,maxlen=max_length,padding='pre',truncating='pre')


In [ ]:
train_y.shape,train_pad_seq.shape,test_pad_seq.shape,test_y.shape

In [ ]:
model=keras.models.Sequential()
model.add(Embedding(total_words,200,input_length=max_length))
#model.add(GlobalAveragePooling1D())
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
adam=keras.optimizers.Adam(learning_rate=0.005)
model.compile(optimizer=adam,loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_pad_seq,train_y,epochs=5)

In [ ]:
encoder_url='https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4'
preprocess_url='https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3'

In [ ]:
bert_encoder=hub.KerasLayer(encoder_url)
bert_preprocess=hub.KerasLayer(preprocess_url)

In [ ]:
input_section=keras.layers.Input(shape=(),dtype=tf.string,name='text')
preprocess_output=bert_preprocess(input_section)
encoded_output=bert_encoder(preprocess_output)
layer1=Dropout(0.1,name='dropout')(encoded_output['pooled_output'])
layer2=Dense(1,activation='sigmoid',name='output')(layer1)

model=keras.Model(inputs=[input_section],outputs=[layer2])

Metrics=[keras.metrics.Accuracy(name='accuracy'),
         keras.metrics.Precision(name='precision'),
         keras.metrics.Recall(name='recall')]
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics='accuracy')
model.fit(train_text,train_y,epochs=10)

In [ ]:
train_X